# Genetic algorithm model calibration

This notebook applies a genetic algorithm to calibrate wall decay coefficients using data from each water quality sensing period.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors
default_colors = plotly.colors.qualitative.Plotly
from bayesian_wq_calibration.simulation import model_simulation, sensor_model_id
from bayesian_wq_calibration.ga import fitness
from bayesian_wq_calibration.plotting import plot_network
from bayesian_wq_calibration.constants import TIMESERIES_DIR

Load sensing data for selected sensing period.

In [ ]:
data_period = 18 # 19 calibration events (as at 30 September 2024)
try:
    flow_df = pd.read_csv(TIMESERIES_DIR / f"processed/{str(data_period).zfill(2)}-flow.csv")
    pressure_df = pd.read_csv(TIMESERIES_DIR / f"processed/{str(data_period).zfill(2)}-pressure.csv")
    wq_df = pd.read_csv(TIMESERIES_DIR / f"processed/{str(data_period).zfill(2)}-wq.csv", low_memory=False)
    cl_df = wq_df[wq_df['data_type'] == 'chlorine']

    print(f"Data period: {data_period}")
    print(f"Start datetime: {flow_df['datetime'].unique()[0]}")
    print(f"End datetime: {flow_df['datetime'].unique()[-1]}")
except:
    print(f"Data period {data_period} does not exist.")

Initialize wall decay coefficients.

In [ ]:
wall_grouping = 'single'
wall_coeff_0 = -0.05
wall_coeffs = None

if wall_grouping == 'single':
    wall_coeffs = [wall_coeff_0] * 1
elif wall_grouping == 'diameter-based':
    wall_coeffs = [wall_coeff_0] * 4
elif wall_grouping == 'roughness-based':
    wall_coeffs = [wall_coeff_0] * 8
elif wall_grouping == 'material-based':
    wall_coeffs = [wall_coeff_0] * 3

Compute initial fitness function value.

In [ ]:
obj_function = 'mse' # 'mse', 'rmse', 'mae', 'mape'
demand_resolution = 'dma' # 'dma', 'wwmd'

obj_0 = fitness(flow_df, pressure_df, cl_df, wall_coeffs, obj_function=obj_function, wall_grouping=wall_grouping, demand_resolution=demand_resolution)